# rosalie dev

Notebook purpose:

- Develop `rosalie` package and templates for working with it

In [5]:
import logging
import os

import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

import rosalie as ro
import fabutils as fu


pd.set_option('display.max_rows', 500)

%config InlineBackend.figure_format ='retina'
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = fu.dgp.dgp_cuped().make_data(n=1_000_000, seed=2312).set_index('id', drop=False)
df.head()

,id,y,x,d
id,,,,
0,0,23.042244,16.416167,1
1,1,22.201519,17.460931,1
2,2,25.232494,22.942863,1
3,3,20.757400,17.170929,1
4,4,21.619749,17.922234,1


In [9]:
def welch_t_test(df, metric, d="d"):
    """Return p-value of Welch's t-test."""
    control_sample = df.query("assignments == 'control'")[metric]
    variant_sample = df.query("assignments == 'treatment'")[metric]
    return ttest_ind(control_sample, variant_sample, equal_var=False).pvalue

def mww(df, metric, d="d"):
    """Return p-value of Mann-Whitney-Wilcoxson U-test."""
    control_sample = df.query("assignments == 'control'")[metric]
    variant_sample = df.query("assignments == 'treatment'")[metric]
    return mannwhitneyu(control_sample, variant_sample).pvalue


## testing

In [118]:
SAMPLE_MIN = 100
SAMPLE_MAX = 700_000
NUM_STEPS = 5
NUM_RUNS = 50
MDES = [0.02]
EVALUATORS = [welch_t_test]

Status quo

In [119]:
%%time


eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=EVALUATORS,
    sample_min=SAMPLE_MIN,
    sample_max=SAMPLE_MAX,
    num_steps=NUM_STEPS,
    mdes=MDES,
    num_runs=NUM_RUNS,
    verbose=False
)
result = eval.run()

 60%|██████    | 3/5 [02:03<01:22, 41.19s/it]


KeyboardInterrupt: 

New

In [123]:
%%time

eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=EVALUATORS,
    sample_min=SAMPLE_MIN,
    sample_max=SAMPLE_MAX,
    num_steps=NUM_STEPS,
    num_runs=NUM_RUNS,
    mdes=MDES,
    verbose=False,
)
result = eval.run()

 40%|████      | 2/5 [00:30<00:45, 15.22s/it]


KeyboardInterrupt: 

## dev

In [11]:
%lprun -f eval.run eval.run()

eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=[welch_t_test],
    sample_min=100,
    sample_max=1_000,
    num_steps=4,
    mdes=[0.02],
    num_runs=20,
    verbose=True,
)
result = eval.run()

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:01<00:00,  3.53it/s]
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
100%|██████████| 4/4 [00:00<00:00,  5.85it/s]


Timer unit: 1e-09 s

Total time: 1.14169 s
File: /Users/fgu/dev/personal/rosalie/rosalie/simulator.py
Function: run at line 118

Line #      Hits         Time  Per Hit   % Time  Line Contents
   118                                               def run(self):
   119                                                   """Run simulation."""
   120         1          0.0      0.0      0.0          results = []
   121         2       2000.0   1000.0      0.0          for preprocessor in self.preprocessors:
   122         1       4000.0   4000.0      0.0              df_preproc = preprocessor(self.df)
   123         5    7659000.0    2e+06      0.7              for sample_size in tqdm(self._calc_sample_sizes()):
   124         4      26000.0   6500.0      0.0                  sample_func = self._get_sample_func()
   125         4    5559000.0    1e+06      0.5                  df_sample = sample_func(df_preproc, sample_size)
   126        84      48000.0    571.4      0.0                  for